## NIPS Topic Model using Expectation Maximization (EM)

The UCI Machine Learning dataset repository hosts several datasets recording word counts for documents [here](https://archive.ics.uci.edu/ml/datasets/Bag+of+Words). Here we will use the NIPS dataset.

It provides (a) a table of word counts per document and (b) a vocabulary list for this dataset at the link.

We implement the multinomial mixture of topics model using our own EM clustering code.

### Cluster to 30 topics, using a simple mixture of multinomial topic model.

In [64]:
# import libs
import numpy as np
import matplotlib.pyplot as plt
import sys
import csv

from math import log
from scipy.sparse import csr_matrix
from scipy.misc import logsumexp as LSE

# read data
D = 1500
W = 12419
NNZ = 746316
J = 30 # number of topics/ clusters
CONVERGENCE_THRESHOLD = 0.1
SMOOTHING_CONST = 0.0002

data = np.loadtxt(r'data/docword.nips.txt', dtype=int, delimiter=' ',skiprows=3)

We use CSR matrix for optimal performance as the data matrix is sparse

In [65]:
# store data as numpy matrix
# we subtract by 1 to make data zero-indexed
row = data[:, 0] - 1
col = data[:, 1] - 1
values = data[:, 2]

x = csr_matrix((values, (row, col)), shape=(D, W))

Here we start with uniform distribution of p and pi.
Rather a better way might be to use results from k-means clustering (a variant of EM) to get rough clusters.

**Caution**: pi should not have a zero-element. An infinitesimal smoothing must be applied in such situations, else no documents may be assigned to the corresponding topic.

In [66]:
# p corresponds to probability of word in a topic
p = np.ones((J, W))
p = 1.0/W * p

# pi corresponds to probability that document belongs to topic
pi = np.ones(J)
pi = 1.0/J * pi

def logsumexp(X):
    """
    log sum exp trick for approximation to avoid underflow/ overflow
    :param X: data matrix
    :return: log sum exp applied to each row of input matrix X
    """
    x_max = X.max(1)
    return x_max + np.log(np.exp(X - x_max[:, None]).sum(1))

In the *E-Step*, we compute:

In the *M-Step*, we compute:

We keep a list of expectation value *q* so we can visualize change in q as we iterate through *EM*

*Note: There are several ways to check for convergences of EM. We use difference between expectations in consecutive iterations as measure of convergence* 

In [77]:
# EM
iter_count = 1
list_of_q = [sys.maxsize]

while True:
    # log likelihood
    ll = x.dot(np.log(p).T) + np.log(pi)

    # calculate w_i,j matrix
    w = np.zeros((D, J))
    row_max = np.amax(ll, 1)
    terms = logsumexp((ll.T - row_max).T)
    sub_array = row_max - terms
    w = np.array([ll[i,] - sub_array[i] for i in range(ll.shape[0])])    
    w = np.exp(w)
    
    # normalize each row of w to sum to 1
    for row_idx in range(w.shape[0]):
        w[row_idx,] = w[row_idx,] / np.sum(w[row_idx,])

    ### E-Step ###
    q = np.sum(ll * w)
    print(q)
    list_of_q.append(q)
    
    # check for convergence
    if abs(q - list_of_q[-2]) < CONVERGENCE_THRESHOLD:
        break
    
    ### M-Step ###
    # update p
    """
    Reference for optimization
    Also use CSR Matrix
      for(j in seq(topics)){
        #Update p's with additive smoothing
        top <- colSums(vecs * wijs[,j]) + smoothing_constant
        bottom <- sum(rowSums(vecs) * wijs[,j]) + (smoothing_constant * num_vocab_words)
        probs[j,] <-top/bottom
    }
    """
    for j in range(J):
        # reset n, d
        n = np.zeros(W)
        d = 0.0
        
        for i in range(D):
            X = x.toarray()
            n += X[i,] * w[i,j]
            d += np.sum(X[i,]) * w[i,j]
            
        p[j,] = (n + SMOOTHING_CONST)/ (d + (SMOOTHING_CONST * W))
          
    # update pi
    pi = np.sum(w, 0)/ D
    
    print("finished iteration", iter_count)
    iter_count += 1

-18221473.4858
0


KeyboardInterrupt: 

### Graph showing, for each topic, the probability with which the topic is selected.


### Table showing, for each topic, the 10 words with the highest probability for that topic.